In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

## Train

In [5]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)

for col in categorical_features:
    dt = df_train[col].dtype
    if dt == int or dt == float:
        df_train[col].fillna(-1, inplace=True)
        df_test[col].fillna(-1, inplace=True)
    else:
        df_train[col].fillna('', inplace=True)
        df_test[col].fillna('', inplace=True)
        
for col in categorical_features:
    if col not in ['pj_no']:
        vc = dict(df_train[col].value_counts())
        for k, v in vc.items():
            if v <= 3:
                df_train[col].replace(k, 'Low_freq', inplace=True)
                df_test[col].replace(k, 'Low_freq', inplace=True)
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col])
        mapping = {v: i for i, v in enumerate(le.classes_)}
        df_test[col] = df_test[col].apply(lambda x: mapping[x] if x in mapping else -1)

## FFM

In [6]:
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
continue_features.remove('keiyaku_pr')
df_train = df_train[['keiyaku_pr']+continue_features+categorical_features]
df_test = df_test[['keiyaku_pr']+continue_features+categorical_features]

In [7]:
def dump_df_in_libsvm_format_with_numeric(row, f):
    line_to_dump = [np.log(row['keiyaku_pr']+1)]
    for feat_idx, (col_name, value) in enumerate(row.items()):
        if col_name in ['keiyaku_pr']:
            continue
        if pd.isnull(value):
            continue
        if col_name in categorical_features:
            line_to_dump.append('%s' % int(value))
            continue
        if col_name in continue_features:
            line_to_dump.append('%.8f' % value)
            continue
        if type(value) == float:
            value = int(value)
        line_to_dump.append('%s' % value)
    line_to_dump = map(str, line_to_dump)
    f.write('%s\n' % ' '.join(line_to_dump))

In [8]:
for i, (train_idx, valid_idx) in enumerate(KFold(n_splits=5, shuffle=True, random_state=28).split(df_train)):
    f_train = open('./data/train_ffm_%d.txt' % i, 'w')
    f_valid = open('./data/valid_ffm_%d.txt' % i, 'w')
    df_train.iloc[train_idx].apply(lambda row: dump_df_in_libsvm_format_with_numeric(row, f_train), axis=1)
    df_train.iloc[valid_idx].apply(lambda row: dump_df_in_libsvm_format_with_numeric(row, f_valid), axis=1)
    f_train.close()
    f_valid.close()

f_test = open('./data/test_ffm.txt', 'w')
df_test.apply(lambda row: dump_df_in_libsvm_format_with_numeric(row, f_test), axis=1)
f_test.close()

In [12]:
f_script = open('./train_ffm_with_numeric_script.py', 'w')

script = '''
import xlearn as xl

for i in range(5):
    ffm_model = xl.create_ffm()
    ffm_model.setTrain('./data/train_ffm_%d.txt' % i)
    ffm_model.setValidate('./data/valid_ffm_%d.txt' % i)
    param = {'task': 'reg', 'lr': 0.3, 'lambda': 0.002, 'epoch': 500, 'stop_window': 30, 'k': 8}

    ffm_model.fit(param, "./model_ffm_%d.out" % i)
    ffm_model.setTest("./data/test_ffm.txt")
    ffm_model.predict("./model_ffm_%d.out" % i, "./pred_ffm_%d.txt" % i)
'''
f_script.write(script)
f_script.close()

In [ ]:
!python3 ./train_ffm_with_numeric_script.py

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 16 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (./data/train_ffm_0.txt.bin) NOT found. Convert text file to binary file.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (./data/valid_ffm_0.txt.bin) NOT found. Convert text file to binary file.
[------------] Number of Feature: 162
[------------] Number of Field: 1
[------------] Time cost for reading problem: 

In [37]:
pred = []
for i in range(5):
    pred.append(np.exp([float(line) for line in open('./pred_ffm_%d.txt' % i)])-1)
pred_ffm = np.mean(pred, 0)
df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = pred_ffm
df_submission.to_csv('submission_ffm.tsv', sep='\t', header=None, index=False)

In [6]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01226572

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's l2: 0.0676625
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0669024
[3]	valid_0's l2: 0.0661908
[4]	valid_0's l2: 0.0654758
[5]	valid_0's l2: 0.0648214
[6]	valid_0's l2: 0.0641902
[7]	valid_0's l2: 0.0634881
[8]	valid_0's l2: 0.0628317
[9]	valid_0's l2: 0.0621559
[10]	valid_0's l2: 0.0615044
[11]	valid_0's l2: 0.060871
[12]	valid_0's l2: 0.0602352
[13]	valid_0's l2: 0.0596189
[14]	valid_0's l2: 0.0590747
[15]	valid_0's l2: 0.0584739
[16]	valid_0's l2: 0.0579306
[17]	valid_0's l2: 0.0573368
[18]	valid_0's l2: 0.0567609
[19]	valid_0's l2: 0.0561873
[20]	valid_0's l2: 0.0556347
[21]	valid_0's l2: 0.0551091
[22]	valid_0's l2: 0.0545774
[23]	valid_0's l2: 0.0540887
[24]	valid_0's l2: 0.0536192
[25]	valid_0's l2: 0.0531193
[26]	valid_0's l2: 0.0526386
[27]	valid_0's l2: 0.0521617
[28]	valid_0's l2: 0.0516798
[29]	valid_0's l2: 0.0512122
[30]	valid_0's l2: 0.0507432
[31]	valid_0's l2: 0.0503182
[32]	valid_0's l2: 0.0498853
[33]	valid_0's l2: 

[323]	valid_0's l2: 0.0203028
[324]	valid_0's l2: 0.0202871
[325]	valid_0's l2: 0.0202736
[326]	valid_0's l2: 0.0202621
[327]	valid_0's l2: 0.0202474
[328]	valid_0's l2: 0.0202342
[329]	valid_0's l2: 0.0202148
[330]	valid_0's l2: 0.0202029
[331]	valid_0's l2: 0.0201901
[332]	valid_0's l2: 0.0201754
[333]	valid_0's l2: 0.0201582
[334]	valid_0's l2: 0.0201418
[335]	valid_0's l2: 0.0201264
[336]	valid_0's l2: 0.0201091
[337]	valid_0's l2: 0.0200907
[338]	valid_0's l2: 0.0200764
[339]	valid_0's l2: 0.020065
[340]	valid_0's l2: 0.0200549
[341]	valid_0's l2: 0.0200391
[342]	valid_0's l2: 0.0200241
[343]	valid_0's l2: 0.0200093
[344]	valid_0's l2: 0.0199968
[345]	valid_0's l2: 0.0199817
[346]	valid_0's l2: 0.019967
[347]	valid_0's l2: 0.0199563
[348]	valid_0's l2: 0.0199422
[349]	valid_0's l2: 0.0199304
[350]	valid_0's l2: 0.0199137
[351]	valid_0's l2: 0.0198979
[352]	valid_0's l2: 0.0198882
[353]	valid_0's l2: 0.019875
[354]	valid_0's l2: 0.0198639
[355]	valid_0's l2: 0.0198581
[356]	valid_0

[653]	valid_0's l2: 0.0179344
[654]	valid_0's l2: 0.0179323
[655]	valid_0's l2: 0.0179291
[656]	valid_0's l2: 0.0179279
[657]	valid_0's l2: 0.0179236
[658]	valid_0's l2: 0.0179217
[659]	valid_0's l2: 0.0179175
[660]	valid_0's l2: 0.0179166
[661]	valid_0's l2: 0.0179127
[662]	valid_0's l2: 0.0179046
[663]	valid_0's l2: 0.0179014
[664]	valid_0's l2: 0.0178992
[665]	valid_0's l2: 0.0178968
[666]	valid_0's l2: 0.0178939
[667]	valid_0's l2: 0.0178945
[668]	valid_0's l2: 0.0178902
[669]	valid_0's l2: 0.0178889
[670]	valid_0's l2: 0.0178876
[671]	valid_0's l2: 0.0178811
[672]	valid_0's l2: 0.0178796
[673]	valid_0's l2: 0.0178742
[674]	valid_0's l2: 0.017874
[675]	valid_0's l2: 0.0178707
[676]	valid_0's l2: 0.0178674
[677]	valid_0's l2: 0.0178666
[678]	valid_0's l2: 0.0178643
[679]	valid_0's l2: 0.0178625
[680]	valid_0's l2: 0.0178577
[681]	valid_0's l2: 0.0178542
[682]	valid_0's l2: 0.0178514
[683]	valid_0's l2: 0.0178473
[684]	valid_0's l2: 0.0178438
[685]	valid_0's l2: 0.0178404
[686]	valid

[994]	valid_0's l2: 0.017363
[995]	valid_0's l2: 0.0173611
[996]	valid_0's l2: 0.0173608
[997]	valid_0's l2: 0.0173592
[998]	valid_0's l2: 0.0173591
[999]	valid_0's l2: 0.0173574
[1000]	valid_0's l2: 0.0173575
[1001]	valid_0's l2: 0.0173568
[1002]	valid_0's l2: 0.0173554
[1003]	valid_0's l2: 0.0173531
[1004]	valid_0's l2: 0.0173522
[1005]	valid_0's l2: 0.0173501
[1006]	valid_0's l2: 0.0173515
[1007]	valid_0's l2: 0.0173505
[1008]	valid_0's l2: 0.0173486
[1009]	valid_0's l2: 0.0173468
[1010]	valid_0's l2: 0.0173443
[1011]	valid_0's l2: 0.0173433
[1012]	valid_0's l2: 0.0173419
[1013]	valid_0's l2: 0.0173434
[1014]	valid_0's l2: 0.0173421
[1015]	valid_0's l2: 0.0173423
[1016]	valid_0's l2: 0.0173417
[1017]	valid_0's l2: 0.0173412
[1018]	valid_0's l2: 0.0173401
[1019]	valid_0's l2: 0.0173399
[1020]	valid_0's l2: 0.0173371
[1021]	valid_0's l2: 0.0173378
[1022]	valid_0's l2: 0.0173338
[1023]	valid_0's l2: 0.0173325
[1024]	valid_0's l2: 0.0173311
[1025]	valid_0's l2: 0.017329
[1026]	valid_0's

...

[1463]	valid_0's l2: 0.0118113
[1464]	valid_0's l2: 0.0118118
[1465]	valid_0's l2: 0.0118122
[1466]	valid_0's l2: 0.0118112
[1467]	valid_0's l2: 0.0118111
[1468]	valid_0's l2: 0.01181
[1469]	valid_0's l2: 0.0118099
[1470]	valid_0's l2: 0.01181
[1471]	valid_0's l2: 0.0118096
[1472]	valid_0's l2: 0.0118078
[1473]	valid_0's l2: 0.0118074
[1474]	valid_0's l2: 0.0118065
[1475]	valid_0's l2: 0.0118086
[1476]	valid_0's l2: 0.0118068
[1477]	valid_0's l2: 0.0118072
[1478]	valid_0's l2: 0.0118069
[1479]	valid_0's l2: 0.0118075
[1480]	valid_0's l2: 0.0118067
[1481]	valid_0's l2: 0.0118072
[1482]	valid_0's l2: 0.0118079
[1483]	valid_0's l2: 0.0118094
[1484]	valid_0's l2: 0.0118093
[1485]	valid_0's l2: 0.0118071
[1486]	valid_0's l2: 0.0118078
[1487]	valid_0's l2: 0.0118086
[1488]	valid_0's l2: 0.0118081
[1489]	valid_0's l2: 0.0118076
[1490]	valid_0's l2: 0.011808
[1491]	valid_0's l2: 0.0118071
[1492]	valid_0's l2: 0.0118076
[1493]	valid_0's l2: 0.0118069
[1494]	valid_0's l2: 0.0118076
[1495]	valid_

[1744]	valid_0's l2: 0.0117455
[1745]	valid_0's l2: 0.0117467
[1746]	valid_0's l2: 0.0117472
[1747]	valid_0's l2: 0.0117476
[1748]	valid_0's l2: 0.0117469
[1749]	valid_0's l2: 0.0117478
[1750]	valid_0's l2: 0.0117472
[1751]	valid_0's l2: 0.0117464
[1752]	valid_0's l2: 0.0117456
[1753]	valid_0's l2: 0.0117441
[1754]	valid_0's l2: 0.0117427
[1755]	valid_0's l2: 0.0117409
[1756]	valid_0's l2: 0.0117423
[1757]	valid_0's l2: 0.0117422
[1758]	valid_0's l2: 0.0117425
[1759]	valid_0's l2: 0.0117403
[1760]	valid_0's l2: 0.0117404
[1761]	valid_0's l2: 0.0117415
[1762]	valid_0's l2: 0.0117408
[1763]	valid_0's l2: 0.0117406
[1764]	valid_0's l2: 0.0117411
[1765]	valid_0's l2: 0.0117384
[1766]	valid_0's l2: 0.0117381
[1767]	valid_0's l2: 0.0117375
[1768]	valid_0's l2: 0.0117378
[1769]	valid_0's l2: 0.0117378
[1770]	valid_0's l2: 0.0117387
[1771]	valid_0's l2: 0.0117396
[1772]	valid_0's l2: 0.0117403
[1773]	valid_0's l2: 0.0117389
[1774]	valid_0's l2: 0.0117391
[1775]	valid_0's l2: 0.01174
[1776]	val

[2031]	valid_0's l2: 0.0116898
[2032]	valid_0's l2: 0.0116897
[2033]	valid_0's l2: 0.0116889
[2034]	valid_0's l2: 0.0116884
[2035]	valid_0's l2: 0.0116892
[2036]	valid_0's l2: 0.0116898
[2037]	valid_0's l2: 0.0116877
[2038]	valid_0's l2: 0.011687
[2039]	valid_0's l2: 0.0116864
[2040]	valid_0's l2: 0.0116859
[2041]	valid_0's l2: 0.0116856
[2042]	valid_0's l2: 0.0116858
[2043]	valid_0's l2: 0.0116859
[2044]	valid_0's l2: 0.011686
[2045]	valid_0's l2: 0.0116859
[2046]	valid_0's l2: 0.0116866
[2047]	valid_0's l2: 0.0116869
[2048]	valid_0's l2: 0.0116867
[2049]	valid_0's l2: 0.0116861
[2050]	valid_0's l2: 0.0116859
[2051]	valid_0's l2: 0.0116847
[2052]	valid_0's l2: 0.0116841
[2053]	valid_0's l2: 0.0116836
[2054]	valid_0's l2: 0.0116836
[2055]	valid_0's l2: 0.0116833
[2056]	valid_0's l2: 0.0116818
[2057]	valid_0's l2: 0.0116813
[2058]	valid_0's l2: 0.0116799
[2059]	valid_0's l2: 0.0116796
[2060]	valid_0's l2: 0.0116794
[2061]	valid_0's l2: 0.011678
[2062]	valid_0's l2: 0.011678
[2063]	valid

[2317]	valid_0's l2: 0.0116662
[2318]	valid_0's l2: 0.0116654
[2319]	valid_0's l2: 0.011664
[2320]	valid_0's l2: 0.0116637
[2321]	valid_0's l2: 0.0116638
[2322]	valid_0's l2: 0.0116641
[2323]	valid_0's l2: 0.0116646
[2324]	valid_0's l2: 0.011664
[2325]	valid_0's l2: 0.0116641
[2326]	valid_0's l2: 0.0116634
[2327]	valid_0's l2: 0.0116633
[2328]	valid_0's l2: 0.0116621
[2329]	valid_0's l2: 0.0116607
[2330]	valid_0's l2: 0.0116608
[2331]	valid_0's l2: 0.0116601
[2332]	valid_0's l2: 0.0116587
[2333]	valid_0's l2: 0.011658
[2334]	valid_0's l2: 0.0116576
[2335]	valid_0's l2: 0.011658
[2336]	valid_0's l2: 0.0116582
[2337]	valid_0's l2: 0.011659
[2338]	valid_0's l2: 0.0116585
[2339]	valid_0's l2: 0.0116579
[2340]	valid_0's l2: 0.0116576
[2341]	valid_0's l2: 0.0116584
[2342]	valid_0's l2: 0.0116581
[2343]	valid_0's l2: 0.0116584
[2344]	valid_0's l2: 0.0116591
[2345]	valid_0's l2: 0.0116602
[2346]	valid_0's l2: 0.0116603
[2347]	valid_0's l2: 0.01166
[2348]	valid_0's l2: 0.0116601
[2349]	valid_0'

[2610]	valid_0's l2: 0.0116684
[2611]	valid_0's l2: 0.0116677
[2612]	valid_0's l2: 0.0116677
[2613]	valid_0's l2: 0.0116678
[2614]	valid_0's l2: 0.0116697
[2615]	valid_0's l2: 0.0116701
[2616]	valid_0's l2: 0.0116708
[2617]	valid_0's l2: 0.0116709
[2618]	valid_0's l2: 0.0116716
[2619]	valid_0's l2: 0.0116715
[2620]	valid_0's l2: 0.0116717
[2621]	valid_0's l2: 0.0116722
[2622]	valid_0's l2: 0.0116725
[2623]	valid_0's l2: 0.0116726
[2624]	valid_0's l2: 0.0116727
[2625]	valid_0's l2: 0.0116729
[2626]	valid_0's l2: 0.0116736
[2627]	valid_0's l2: 0.0116731
[2628]	valid_0's l2: 0.0116716
[2629]	valid_0's l2: 0.0116713
[2630]	valid_0's l2: 0.0116708
[2631]	valid_0's l2: 0.0116712
[2632]	valid_0's l2: 0.0116713
[2633]	valid_0's l2: 0.0116714
[2634]	valid_0's l2: 0.0116713
[2635]	valid_0's l2: 0.011671
[2636]	valid_0's l2: 0.0116706
[2637]	valid_0's l2: 0.0116709
[2638]	valid_0's l2: 0.0116705
[2639]	valid_0's l2: 0.01167
[2640]	valid_0's l2: 0.0116707
[2641]	valid_0's l2: 0.0116701
[2642]	vali